In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import pprint
import torch.nn as nn

import matplotlib.pyplot as plt
%matplotlib inline

# for creating validation set
from sklearn.model_selection import train_test_split

# for evaluating the model
from sklearn.metrics import accuracy_score

# PyTorch libraries and modules
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv1d, Conv2d,MaxPool1d, MaxPool2d, Module, Softmax, BatchNorm1d, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR

# torchvision for pre-trained models
from torchvision import models

import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torchvision import transforms


In [2]:
# # Check if CUDA is available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device


In [3]:
master_csv = pd.read_csv("MASTER_CSV_4.csv")
master_csv.head()
master_csv.shape


(12660, 4)

In [4]:
cwd = os.getcwd() 
folder_path = os.path.join(cwd,"session_1")

# eeg_data = []
# for filename in tqdm(master_csv['filename']):
#     file_path = os.path.join(folder_path, filename)
#     file = pd.read_csv(file_path)
    
#     file = file.astype('float32')
    
#     # Calculate mean and std
#     mean, std = file.mean(), file.std()
    
#     # Normalize EEG data
#     normalized_data = (file - mean) / std
    
#     eeg_data.append(normalized_data)

# # Convert the list of normalized arrays to a NumPy array
# eeg_numpy = np.array(eeg_data)


eeg_numpy = np.load('eeg_numpy.npy')
print(eeg_numpy.shape)


(12660, 1001, 62)


In [5]:
#eeg_data

In [14]:
eeg_numpy

array([[[ 1.1158739 ,  1.236367  ,  1.2142688 , ...,  1.4467032 ,
          1.6951935 ,  1.7026098 ],
        [ 1.6328441 ,  1.443384  ,  1.3650314 , ...,  1.7051029 ,
          1.8625292 ,  1.8763801 ],
        [ 1.2299114 ,  1.7298721 ,  1.1660248 , ...,  1.3870724 ,
          1.5190508 ,  1.5187662 ],
        ...,
        [-0.9697455 , -1.283159  , -1.5356385 , ..., -1.0816393 ,
         -0.64247245, -0.60677016],
        [-1.1737459 , -1.7648908 , -1.7635913 , ..., -1.114436  ,
         -0.46003878, -0.43174088],
        [-0.9127267 , -1.3714602 , -1.0230461 , ..., -1.2674884 ,
         -0.83622926, -0.8774991 ]],

       [[ 2.528494  ,  2.4359193 ,  2.4094703 , ...,  0.3816055 ,
          0.35974196,  0.43337438],
        [ 3.455898  ,  3.6334217 ,  3.503844  , ..., -0.30513126,
         -0.35826373, -0.38696492],
        [ 4.0914726 ,  4.528642  ,  4.5213394 , ...,  0.5561996 ,
          0.42966947,  0.41957662],
        ...,
        [-0.98761904, -0.9482108 , -0.93605655, ...,  

In [15]:
eeg_numpy.shape # 360, 400, 62

(12660, 1001, 62)

In [16]:
# defining the target
label_emotion = master_csv['emotion'].values
print(label_emotion[:48])  # Display the first 10 elements

[2 2 3 3 0 3 0 1 0 2 1 3 1 3 1 1 0 2 2 2 2 3 1 2 0 3 3 3 3 1 2 1 1 0 3 3 0
 0 1 2 2 3 0 2 1 3 3 1]


In [17]:
# create validation set
train_x, val_x, train_y, val_y = train_test_split(eeg_numpy, label_emotion, test_size = 0.25, random_state = 13, stratify=label_emotion)
(train_x.shape, train_y.shape), (val_x.shape, val_y.shape)

(((9495, 1001, 62), (9495,)), ((3165, 1001, 62), (3165,)))

In [18]:
# converting training images into torch format
train_x = train_x.reshape(9495, 1, 1001, 62)
train_x  = torch.from_numpy(train_x)

# converting the target into torch format
train_y = train_y.astype(int)
train_y = torch.from_numpy(train_y).long()

# shape of training data
train_x.shape, train_y.shape

# converting validation images into torch format
val_x = val_x.reshape(3165, 1, 1001, 62)
val_x  = torch.from_numpy(val_x)

# converting the target into torch format
val_y = val_y.astype(int)
val_y = torch.from_numpy(val_y).long()

# shape of validation data
val_x.shape, val_y.shape

(torch.Size([3165, 1, 1001, 62]), torch.Size([3165]))

In [19]:
class ConvNetGray(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
        )
        self.fc = nn.Sequential(
            nn.Linear(512 * 62 * 3, 256),
            # nn.Linear(512 * 12 * 1, 256),  # Adjusted to match the output from convolutional layers
            nn.Linear(256, 128),
            nn.Linear(128, 64),
            nn.Linear(64, 4)
        )

    # def forward(self, x):
    #     out = self.conv2(x)
    #     out = out.view(out.size(0), -1)  # Flatten the output
    #     out = self.fc(out)
    #     return out

    def forward(self, x):
        out = self.conv2(x)
        print(f"Output shape after convolutions: {out.shape}")  # This will print the output shape
        out = out.view(out.size(0), -1)  # Flatten the output
        out = self.fc(out)
        return out



In [20]:
# defining the model
model = ConvNetGray()
# defining the optimizer
optimizer = Adam(model.parameters(), lr=0.0001)
# defining the loss function
criterion = CrossEntropyLoss()
device = torch.device("cpu")
model = model.to(device)
criterion = criterion.to(device)

print(model)

ConvNetGray(
  (conv2): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
    (7): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (8): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU()
    (10): MaxPool2d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (12): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU()
    (14): MaxPool2d(kernel_size=3, stride=1, padding=1, dilation=

In [ ]:
%%time
# batch size of the model
batch_size = 40

# Define parameters
n_epochs = 20  # Define the maximum number of epochs
early_stop_patience = 10  # Define patience for early stopping
min_loss = float('inf')  # Initialize minimum loss
best_epoch = 0  # Initialize the epoch with the best performance
no_improvement_count = 0  # Initialize the count for early stopping

train_losses = []  # List to store training losses

# number of epochs to train the model
#n_epochs = 15

for epoch in range(1, n_epochs+1):

    # keep track of training and validation loss
    train_loss = 0.0
        
    permutation = torch.randperm(train_x.size()[0])
    #permutation = torch.randperm(train_x.size()[0])

    training_loss = []
    for i in tqdm(range(0,train_x.size()[0], batch_size)):

        indices = permutation[i:i+batch_size]
        batch_x, batch_y = train_x[indices], train_y[indices]
        
        #if torch.cuda.is_available():
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        
        optimizer.zero_grad()
        # in case you wanted a semi-full example
        outputs = model(batch_x)
        loss = criterion(outputs,batch_y)

        training_loss.append(loss.item())
        loss.backward()
        optimizer.step()
        train_losses.append(training_loss)

    training_loss = np.average(training_loss)
    print('epoch: \t', epoch, '\t training loss: \t', training_loss)

    # Check for early stopping
    if training_loss < min_loss:
        min_loss = training_loss
        best_epoch = epoch
        no_improvement_count = 0
    else:
        no_improvement_count += 1

    if no_improvement_count >= early_stop_patience:
        print(f'Early stopping at epoch {epoch} with minimum loss {min_loss}')
        break

  0%|                                                                                          | 0/238 [00:00<?, ?it/s]

Output shape after convolutions: torch.Size([40, 512, 62, 3])


  0%|▎                                                                                 | 1/238 [00:08<33:51,  8.57s/it]

Output shape after convolutions: torch.Size([40, 512, 62, 3])


  1%|▋                                                                                 | 2/238 [00:15<29:53,  7.60s/it]

...

 13%|██████████▌                                                                      | 31/238 [04:25<27:07,  7.86s/it]

Output shape after convolutions: torch.Size([40, 512, 62, 3])


 13%|██████████▉                                                                      | 32/238 [04:32<26:16,  7.65s/it]

Output shape after convolutions: torch.Size([40, 512, 62, 3])


 14%|███████████▏                                                                     | 33/238 [04:39<25:36,  7.50s/it]

In [15]:
%%time
# prediction for training set
prediction = []
target = []
permutation = torch.randperm(train_x.size()[0])
for i in tqdm(range(0,train_x.size()[0], batch_size)):
    indices = permutation[i:i+batch_size]
    batch_x, batch_y = train_x[indices], train_y[indices]

    #if torch.cuda.is_available():
    batch_x, batch_y = batch_x.to(device), batch_y.to(device)

    with torch.no_grad():
        output = model(batch_x)

    softmax = torch.exp(output).cpu()
    prob = list(softmax.numpy())
    predictions = np.argmax(prob, axis=1)
    prediction.append(predictions)
    target.append(batch_y)
    
# training accuracy
accuracy = []
for i in range(len(prediction)):
    accuracy.append(accuracy_score(target[i],prediction[i]))
    
print('training accuracy: \t', np.average(accuracy))


  0%|                                                                                          | 0/238 [00:00<?, ?it/s]


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [150]:
%%time
# prediction for validation set
prediction_val = []
target_val = []
permutation = torch.randperm(val_x.size()[0])
for i in tqdm(range(0,val_x.size()[0], batch_size)):
    indices = permutation[i:i+batch_size]
    batch_x, batch_y = val_x[indices], val_y[indices]

    #if torch.cuda.is_available():
    batch_x, batch_y = batch_x.to(device), batch_y.to(device)

    with torch.no_grad():
        output = model(batch_x)

    softmax = torch.exp(output).cpu()
    prob = list(softmax.numpy())
    predictions = np.argmax(prob, axis=1)
    prediction_val.append(predictions)
    target_val.append(batch_y)
    
# validation accuracy
accuracy_val = []
for i in range(len(prediction_val)):
    accuracy_val.append(accuracy_score(target_val[i],prediction_val[i]))
    
print('validation accuracy: \t', np.average(accuracy_val))

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:08<00:00,  8.73s/it]

validation accuracy: 	 0.34444444444444444
